In [2]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

#read in our data
crashes_df = pd.read_csv("..\Resources\Road_Crashes_for_five_Years_Victoria.csv")
crashes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60689 entries, 0 to 60688
Data columns (total 65 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   X                  60689 non-null  float64
 1   Y                  60689 non-null  float64
 2   OBJECTID           60689 non-null  int64  
 3   ACCIDENT_NO        60689 non-null  object 
 4   ABS_CODE           60689 non-null  object 
 5   ACCIDENT_STATUS    60689 non-null  object 
 6   ACCIDENT_DATE      60689 non-null  object 
 7   ACCIDENT_TIME      60689 non-null  object 
 8   ALCOHOLTIME        60689 non-null  object 
 9   ACCIDENT_TYPE      60689 non-null  object 
 10  DAY_OF_WEEK        60689 non-null  int64  
 11  DCA_CODE           60689 non-null  object 
 12  HIT_RUN_FLAG       60689 non-null  object 
 13  LIGHT_CONDITION    60689 non-null  object 
 14  POLICE_ATTEND      60689 non-null  object 
 15  ROAD_GEOMETRY      60689 non-null  object 
 16  SEVERITY           606

In [3]:
#convert date and time columns to the appropriate data types
crashes_df["ACCIDENT_DATE"] = pd.to_datetime(crashes_df["ACCIDENT_DATE"], dayfirst=True)
crashes_df["ACCIDENT_TIME"] = pd.to_numeric(crashes_df["ACCIDENT_TIME"].str.slice(stop=2))

In [4]:
crashes_df.head()

,X,Y,OBJECTID,ACCIDENT_NO,ABS_CODE,ACCIDENT_STATUS,ACCIDENT_DATE,ACCIDENT_TIME,ALCOHOLTIME,ACCIDENT_TYPE,...,DEG_URBAN_ALL,LGA_NAME_ALL,REGION_NAME_ALL,SRNS,SRNS_ALL,RMA,RMA_ALL,DIVIDED,DIVIDED_ALL,STAT_DIV_NAME
0,2444652.517,2369157.816,1,T20150013818,ABS to receive accident,Finished,2015-07-01,5,Yes,Collision with a fixed object,...,LARGE_PROVINCIAL_CITIES,GEELONG,SOUTH WESTERN REGION,NaN,NaN,Local Road,Local Road,Undivided,Undivided,Country
1,2507053.522,2427188.691,2,T20150013844,ABS to receive accident,Finished,2015-07-01,11,No,Collision with a fixed object,...,MELB_URBAN,WHITTLESEA,METROPOLITAN NORTH WEST REGION,C,C,Arterial Other,"Arterial Other,Local Road",Divided,"Undivided,Divided",Metro
2,2490706.473,2409439.682,3,T20150013854,ABS to receive accident,Finished,2015-07-01,13,No,Collision with vehicle,...,MELB_URBAN,MARIBYRNONG,METROPOLITAN NORTH WEST REGION,NaN,NaN,Local Road,Local Road,Undivided,Undivided,Metro
3,2524370.961,2379929.797,4,T20150013897,Non ABS accident,Finished,2015-07-02,10,No,Struck Pedestrian,...,MELB_URBAN,CASEY,METROPOLITAN SOUTH EAST REGION,NaN,NaN,NaN,NaN,NaN,NaN,Metro
4,2346439.879,2385852.164,5,T20150013907,ABS to receive accident,Finished,2015-07-02,10,No,Vehicle overturned (no collision),...,RURAL_VICTORIA,CORANGAMITE,SOUTH WESTERN REGION,C,C,Arterial Other,Arterial Other,Undivided,Undivided,Country


In [5]:
#drop unnecessary columns, and drop nan values
crashes_df=crashes_df.drop(columns=["X", "Y", "ACCIDENT_NO", "SRNS", "SRNS_ALL", "DIVIDED", "DIVIDED_ALL", "UNKNOWN", "NODE_ID", "OBJECTID"])
crashes_df = crashes_df.dropna()

### Why do we drop these columns?
X & Y are duplicates of the Latitude and Longitude columns, so we don't need those\
ACCIDENT NO, NODE_ID  & OBJECTID are an id numbers for the crash- unnecessary for our purposes\
SRNS and SRNS_ALL - these columns had only 10% of the values filled in over the 60000 rows. We were also unable to determine what it meant. \
DIVIDED and DIVIDED_ALL - again, we were unable to determine what this referred to. DIVIDED_ALL also had values that did not make any sense- ie 'Undivided,Divided'. We suspect this is related to whetehr the road had a median strip or was a dual carriageway, if this is the case we felt this was covered by road_type. \
UNKNOWN is self explanatory

In [6]:
#display the number of unique values in each categorical column
crashes_df[["ABS_CODE", "ACCIDENT_STATUS", "ALCOHOLTIME", "ACCIDENT_TYPE", "DCA_CODE", "HIT_RUN_FLAG", "LIGHT_CONDITION",
            "POLICE_ATTEND", "ROAD_GEOMETRY", "SEVERITY", "SPEED_ZONE", "RUN_OFFROAD", "NODE_TYPE", "LGA_NAME", "REGION_NAME", 
            "ALCOHOL_RELATED", "DEG_URBAN_NAME", "DEG_URBAN_ALL", "LGA_NAME_ALL", "REGION_NAME_ALL", "RMA", "RMA_ALL", "STAT_DIV_NAME"]].nunique()

ABS_CODE             3
ACCIDENT_STATUS      5
ALCOHOLTIME          2
ACCIDENT_TYPE        9
DCA_CODE            81
HIT_RUN_FLAG         3
LIGHT_CONDITION      7
POLICE_ATTEND        3
ROAD_GEOMETRY        9
SEVERITY             4
SPEED_ZONE          13
RUN_OFFROAD          2
NODE_TYPE            2
LGA_NAME            87
REGION_NAME          8
ALCOHOL_RELATED      2
DEG_URBAN_NAME       7
DEG_URBAN_ALL       18
LGA_NAME_ALL       215
REGION_NAME_ALL     18
RMA                  5
RMA_ALL             44
STAT_DIV_NAME        2
dtype: int64

In [7]:
#lets see the values with more than 500 occurances in the DCA_CODE column
values = crashes_df['DCA_CODE'].value_counts()
print(values[values > 500])

DCA_CODE
REAR END(VEHICLES IN SAME LANE)                             10169
RIGHT THROUGH                                                4940
CROSS TRAFFIC(INTERSECTIONS ONLY)                            4189
LEFT OFF CARRIAGEWAY INTO OBJECT/PARKED VEHICLE              3260
RIGHT NEAR (INTERSECTIONS ONLY)                              2536
RIGHT OFF CARRIAGEWAY INTO OBJECT/PARKED VEHICLE             2270
PED NEAR SIDE. PED HIT BY VEHICLE FROM THE RIGHT.            1932
OUT OF CONTROL ON CARRIAGEWAY (ON STRAIGHT)                  1909
HEAD ON (NOT OVERTAKING)                                     1891
OFF RIGHT BEND INTO OBJECT/PARKED VEHICLE                    1676
VEHICLE COLLIDES WITH VEHICLE PARKED ON LEFT OF ROAD         1354
FAR SIDE. PED HIT BY VEHICLE FROM THE LEFT                   1229
OFF LEFT BEND INTO OBJECT/PARKED VEHICLE                     1208
VEHICLE STRIKES ANOTHER VEH WHILE EMERGING FROM DRIVEWAY     1122
U TURN                                                        931
R

In [8]:
#we decide on a value of 500 as our threshold value
DCA_to_replace = ["VEHICLE COLLIDES WITH VEHICLE PARKED ON LEFT OF ROAD",
"FAR SIDE. PED HIT BY VEHICLE FROM THE LEFT",
"OFF LEFT BEND INTO OBJECT/PARKED VEHICLE",
"VEHICLE STRIKES ANOTHER VEH WHILE EMERGING FROM DRIVEWAY",
"U TURN",
"RIGHT REAR.",
"LANE CHANGE LEFT (NOT OVERTAKING)",
"RIGHT FAR (INTERSECTIONS ONLY)",
"LEFT NEAR (INTERSECTIONS ONLY)",
"LANE SIDE SWIPE (VEHICLES IN PARALLEL LANES)",
"VEHICLE STRIKES DOOR OF PARKED/STATIONARY VEHICLE",
"OFF CARRIAGEWAY TO LEFT",
"OUT OF CONTROL ON CARRIAGEWAY (ON BEND)",
"LANE CHANGE RIGHT (NOT OVERTAKING)",
"STRUCK ANIMAL",
"OFF CARRIAGEWAY ON RIGHT BEND",
"LEFT TURN SIDESWIPE",
"OTHER ACCIDENTS-OFF STRAIGHT NOT INCLUDED IN DCAs 170-175",
"LEFT REAR",
"ANY MANOEUVRE INVOLVING PED NOT INCLUDED IN DCAs 100-108.",
"OFF CARRIAGEWAY ON LEFT BEND",
"STRUCK OBJECT ON CARRIAGEWAY",
"OFF CARRIAGEWAY TO RIGHT",
"PED EMERGES FROM IN FRONT OF PARKED OR STATIONARY VEHICLE",
"PED PLAYING/LYING/WORKING/STANDING ON CARRIAGEWAY.",
"RIGHT TURN SIDESWIPE",
"VEHICLE OFF FOOTPATH STRIKES VEH ON CARRIAGEWAY",
"OFF END OF ROAD/T-INTERSECTION.",
"PULLING OUT (OVERTAKING)",
"OTHER SAME DIRECTION-MANOUEVRES NOT INCLUDED IN DCAs 130-137",
"PED ON FOOTHPATH STRUCK BY VEHENTERING/LEAVING DRIVEWAY.",
"FELL IN/FROM VEHICLE",
"LEAVING PARKING",
"OTHER ADJACENT (INTERSECTIONS ONLY)",
"OTHER ON PATH",
"OUT OF CONTROL (OVERTAKING)",
"OTHER MANOEUVRING NOT INCLUDED IN DCAs 140-148",
"VEH STRIKES PED ON FOOTPATH/MEDIAN/TRAFFIC ISLAND.",
"ENTERING PARKING",
"PED STRUCK WALKING TO/FROM OR BOARDING/ALIGHTING VEHICLE.",
"OTHER ACCIDENTS NOT CLASSIFIABLE ELSEWHERE",
"LEFT FAR (INTERSECTIONS ONLY)",
"HEAD ON(OVERTAKING)",
"PED WALKING WITH TRAFFIC",
"TWO RIGHT TURNING (INTERSECTIONS ONLY)",
"PARKED CAR RUN AWAY",
"REVERSING INTO FIXED OBJECT/PARKED VEHICLE",
"CUTTING IN (OVERTAKING)",
"REVERSING IN STREAM OF TRAFFIC",
"OTHER ACCIDENTS ON CURVE NOT INCLUDED IN DCAs 180-184",
"PERMANENT OBSTRUCTION ON CARRIAGEWAY",
"LOAD OR MISSILE STRUCK VEHICLE",
"ACCIDENT OR BROKEN DOWN",
"PULLING OUT -REAR END",
"OTHER OPPOSING MANOEUVRES NOT INCLUDED IN DCAs 120-125.",
"UNKNOWN-NO DETAILS ON MANOEUVRES OF ROAD-USERS IN ACCIDENT",
"U TURN INTO FIXED OBJECT/PARKED VEHICLE",
"RIGHT/LEFT. ONE VEH TURNING RIGHT THE OTHER LEFT.",
"PED WALKING AGAINST TRAFFIC.",
"OTHER OVERTAKING MANOEUVRES NOT INCLUDED IN DCAs 150-154",
"LEFT THROUGH",
"RIGHT/LEFT FAR (INTERSECTIONS ONLY)",
"TEMPORARY ROADWORKS",
"RIGHT/RIGHT BOTH VEHs FROM OPPOSITE DIRECTIONS TURNING RIGHT",
"STRUCK TRAIN",
"PARKED VEHICLES ONLY",
"DOUBLE PARKED", 
"STRUCK RAILWAY CROSSING FURNITURE",
"TWO LEFT TURNING (INTERSECTIONS ONLY)",
"LEFT/LEFT. BOTH VEHs FROM OPPOSITE DIRECTIONS TURNING LEFT.",
"LEFT/RIGHT FAR (INTERSECTIONS ONLY)"]

#bin the above list into 'other'
for DCA in DCA_to_replace:
    crashes_df['DCA_CODE'] = crashes_df['DCA_CODE'].replace(DCA,"Other")

# Check to make sure binning was successful
crashes_df['DCA_CODE'].value_counts()

DCA_CODE
Other                                                21643
REAR END(VEHICLES IN SAME LANE)                      10169
RIGHT THROUGH                                         4940
CROSS TRAFFIC(INTERSECTIONS ONLY)                     4189
LEFT OFF CARRIAGEWAY INTO OBJECT/PARKED VEHICLE       3260
RIGHT NEAR (INTERSECTIONS ONLY)                       2536
RIGHT OFF CARRIAGEWAY INTO OBJECT/PARKED VEHICLE      2270
PED NEAR SIDE. PED HIT BY VEHICLE FROM THE RIGHT.     1932
OUT OF CONTROL ON CARRIAGEWAY (ON STRAIGHT)           1909
HEAD ON (NOT OVERTAKING)                              1891
OFF RIGHT BEND INTO OBJECT/PARKED VEHICLE             1676
Name: count, dtype: int64

In [9]:
#lets see the value-counts for speed_zone
crashes_df["SPEED_ZONE"].value_counts()

SPEED_ZONE
60 km/hr                       18971
50 km/hr                        9085
80 km/hr                        8431
100 km/hr                       8415
40 km/hr                        3501
70 km/hr                        3402
Not known                       3132
110 km/hr                        645
Camping grounds or off road      501
90 km/hr                         149
30km/hr                           88
Other speed limit                 86
75 km/hr                           9
Name: count, dtype: int64

In [10]:
#we decide on a threshold value of 100 to determine binning
speed_to_replace = ["30km/hr", "75 km/hr"]

#replace the above list with other
for speed in speed_to_replace:
    crashes_df['SPEED_ZONE'] = crashes_df['SPEED_ZONE'].replace(speed,"Other speed limit")

# Check to make sure binning was successful
crashes_df['SPEED_ZONE'].value_counts()

SPEED_ZONE
60 km/hr                       18971
50 km/hr                        9085
80 km/hr                        8431
100 km/hr                       8415
40 km/hr                        3501
70 km/hr                        3402
Not known                       3132
110 km/hr                        645
Camping grounds or off road      501
Other speed limit                183
90 km/hr                         149
Name: count, dtype: int64

In [11]:
#lets see the values with mroe than 500 occurances in the LGA_NAME column
values = crashes_df['LGA_NAME'].value_counts()
print(values[values > 500])

LGA_NAME
MELBOURNE               3049
CASEY                   2420
GEELONG                 2273
DANDENONG               1936
HUME                    1870
BRIMBANK                1683
MONASH                  1663
WHITTLESEA              1656
MORELAND                1619
YARRA RANGES            1582
YARRA                   1395
DAREBIN                 1377
KINGSTON                1355
WYNDHAM                 1270
BOROONDARA              1231
WHITEHORSE              1208
MORNINGTON PENINSULA    1163
STONNINGTON             1162
KNOX                    1134
PORT PHILLIP            1103
GLEN EIRA               1083
BENDIGO                 1077
BALLARAT                1040
FRANKSTON               1004
MELTON                   955
MOONEE VALLEY            905
CARDINIA                 882
MAROONDAH                851
BANYULE                  841
MARIBYRNONG              792
HOBSONS BAY              770
MANNINGHAM               732
SHEPPARTON               730
BAYSIDE                  703
BAW B

### Why don't we bin LGA?
We decided not to bin LGA for two reasons; the first being that this is what we are aiming to predict. The second is that the more fequent value only represents 5.5% of the data- if we binned this into 10 bins we lose far too much precision. 

This encourages us to look into the other columns about area to see if they are more appropriate to use in our modeling. 

In [12]:
#lets see the value counts for DEG_URBAN_ALL
crashes_df['DEG_URBAN_ALL'].value_counts()

DEG_URBAN_ALL
MELB_URBAN                                 34928
RURAL_VICTORIA                             11836
LARGE_PROVINCIAL_CITIES                     3220
SMALL_CITIES                                2855
TOWNS                                       1694
SMALL_TOWNS                                  487
MELBOURNE_CBD                                469
MELB_URBAN,RURAL_VICTORIA                    318
MELB_URBAN,MELBOURNE_CBD                     280
RURAL_VICTORIA,SMALL_CITIES                  114
RURAL_VICTORIA,TOWNS                          91
LARGE_PROVINCIAL_CITIES,RURAL_VICTORIA        45
RURAL_VICTORIA,SMALL_TOWNS                    43
MELB_URBAN,SMALL_TOWNS                        27
SMALL_CITIES,SMALL_TOWNS                       3
SMALL_CITIES,RURAL_VICTORIA                    2
RURAL_VICTORIA,MELB_URBAN                      2
RURAL_VICTORIA,SMALL_CITIES,SMALL_TOWNS        1
Name: count, dtype: int64

In [13]:
#We choose to bin all the ones with multiple area listed
urban_to_replace = ["MELB_URBAN,RURAL_VICTORIA",
"MELB_URBAN,MELBOURNE_CBD",
"RURAL_VICTORIA,SMALL_CITIES",
"RURAL_VICTORIA,TOWNS",
"LARGE_PROVINCIAL_CITIES,RURAL_VICTORIA",
"RURAL_VICTORIA,SMALL_TOWNS",
"MELB_URBAN,SMALL_TOWNS",
"SMALL_CITIES,SMALL_TOWNS",
"SMALL_CITIES,RURAL_VICTORIA",
"RURAL_VICTORIA,MELB_URBAN",
"RURAL_VICTORIA,SMALL_CITIES,SMALL_TOWNS"]

for urban in urban_to_replace:
    crashes_df['DEG_URBAN_ALL'] = crashes_df['DEG_URBAN_ALL'].replace(urban,"Other")

# Check to make sure binning was successful
crashes_df['DEG_URBAN_ALL'].value_counts()

DEG_URBAN_ALL
MELB_URBAN                 34928
RURAL_VICTORIA             11836
LARGE_PROVINCIAL_CITIES     3220
SMALL_CITIES                2855
TOWNS                       1694
Other                        926
SMALL_TOWNS                  487
MELBOURNE_CBD                469
Name: count, dtype: int64

In [14]:
#lets see all the value counts for region name all
crashes_df["REGION_NAME_ALL"].value_counts()

REGION_NAME_ALL
METROPOLITAN SOUTH EAST REGION                                   19966
METROPOLITAN NORTH WEST REGION                                   19776
SOUTH WESTERN REGION                                              4315
EASTERN REGION,EASTERN REGION                                     3126
NORTHERN REGION                                                   3048
NORTH EASTERN REGION                                              3035
WESTERN REGION                                                    2914
METROPOLITAN SOUTH EAST REGION,METROPOLITAN NORTH WEST REGION      164
WESTERN REGION,SOUTH WESTERN REGION                                 19
EASTERN REGION,EASTERN REGION,METROPOLITAN SOUTH EAST REGION        16
NORTH EASTERN REGION,METROPOLITAN SOUTH EAST REGION                 11
NORTH EASTERN REGION,NORTHERN REGION                                 8
WESTERN REGION,NORTHERN REGION                                       6
NORTHERN REGION,METROPOLITAN NORTH WEST REGION               

In [15]:
#we choose to bin all the ones with multiple regions listed
regions_to_replace = ["METROPOLITAN SOUTH EAST REGION,METROPOLITAN NORTH WEST REGION",
"WESTERN REGION,SOUTH WESTERN REGION",
"EASTERN REGION,EASTERN REGION,METROPOLITAN SOUTH EAST REGION",
"NORTH EASTERN REGION,METROPOLITAN SOUTH EAST REGION",
"NORTH EASTERN REGION,NORTHERN REGION",
"WESTERN REGION,NORTHERN REGION",
"NORTHERN REGION,METROPOLITAN NORTH WEST REGION",
"EASTERN REGION,EASTERN REGION,NORTH EASTERN REGION",
"NORTH EASTERN REGION,METROPOLITAN NORTH WEST REGION",
"EASTERN REGION,METROPOLITAN SOUTH EAST REGION,EASTERN REGION",
"METROPOLITAN NORTH WEST REGION,METROPOLITAN SOUTH EAST REGION"]

for region in regions_to_replace:
    crashes_df['REGION_NAME_ALL'] = crashes_df['REGION_NAME_ALL'].replace(region,"Other")

# Check to make sure binning was successful
crashes_df['REGION_NAME_ALL'].value_counts()

REGION_NAME_ALL
METROPOLITAN SOUTH EAST REGION    19966
METROPOLITAN NORTH WEST REGION    19776
SOUTH WESTERN REGION               4315
EASTERN REGION,EASTERN REGION      3126
NORTHERN REGION                    3048
NORTH EASTERN REGION               3035
WESTERN REGION                     2914
Other                               235
Name: count, dtype: int64

In [16]:
#lets see all the value counts for rma_all
crashes_df['RMA_ALL'].value_counts()

RMA_ALL
Local Road                                    20032
Arterial Other                                11845
Arterial Other,Local Road                      7606
Arterial Highway                               5500
Freeway                                        3662
Local Road,Arterial Highway                    2757
Local Road,Arterial Other                      1480
Arterial Highway,Local Road                     953
Arterial Other,Arterial Highway                 789
Arterial Highway,Arterial Other                 548
Freeway,Arterial Other                          398
Arterial Other,Local Road,Arterial Highway      201
Arterial Highway,Arterial Other,Local Road       87
Freeway,Local Road                               77
Arterial Other,Freeway                           70
Arterial Highway,Freeway                         69
Freeway,Arterial Highway                         65
Local Road,Arterial Highway,Arterial Other       47
Local Road,Freeway                               44
Free

In [17]:
#our theshold value here is 300
rma_to_replace = ["Arterial Other,Local Road,Arterial Highway",
"Arterial Highway,Arterial Other,Local Road",
"Freeway,Local Road",
"Arterial Other,Freeway",
"Arterial Highway,Freeway",
"Freeway,Arterial Highway",
"Local Road,Arterial Highway,Arterial Other",
"Local Road,Freeway",
"Freeway,Arterial Other,Local Road",
"Local Road,Freeway,Arterial Other",
"Arterial Highway,Freeway,Arterial Other",
"Freeway,Arterial Other,Arterial Highway",
"Arterial Highway,Non Arterial,Local Road",
"Non Arterial,Local Road",
"Local Road,Arterial Other,Arterial Highway",
"Freeway,Local Road,Arterial Highway",
"Non Arterial",
"Non Arterial,Arterial Other",
"Freeway,Local Road,Non Arterial",
"Arterial Other,Local Road,Freeway",
"Freeway,Non Arterial",
"Arterial Other,Freeway,Arterial Highway",
"Non Arterial,Freeway",
"Arterial Other,Freeway,Local Road",
"Non Arterial,Freeway,Local Road",
"Arterial Other,Arterial Highway,Local Road",
"Arterial Other,Non Arterial",
"Local Road,Arterial Highway,Non Arterial",
"Local Road,Arterial Highway,Freeway",
"Arterial Highway,Freeway,Local Road",
"Local Road,Non Arterial,Freeway",
"Local Road,Non Arterial",
"Arterial Other,Arterial Highway,Freeway"]

for rma in rma_to_replace:
    crashes_df['RMA_ALL'] = crashes_df['RMA_ALL'].replace(rma,"Other")

# Check to make sure binning was successful
crashes_df['RMA_ALL'].value_counts()

RMA_ALL
Local Road                         20032
Arterial Other                     11845
Arterial Other,Local Road           7606
Arterial Highway                    5500
Freeway                             3662
Local Road,Arterial Highway         2757
Local Road,Arterial Other           1480
Arterial Highway,Local Road          953
Other                                845
Arterial Other,Arterial Highway      789
Arterial Highway,Arterial Other      548
Freeway,Arterial Other               398
Name: count, dtype: int64

In [18]:
#check that we didn't miss a value that needs binning
crashes_df[["ABS_CODE", "ACCIDENT_STATUS", "ALCOHOLTIME", "ACCIDENT_TYPE", "DCA_CODE", "HIT_RUN_FLAG", "LIGHT_CONDITION",
            "POLICE_ATTEND", "ROAD_GEOMETRY", "SEVERITY", "SPEED_ZONE", "RUN_OFFROAD", "NODE_TYPE", "LGA_NAME", "REGION_NAME", 
            "ALCOHOL_RELATED", "DEG_URBAN_NAME", "DEG_URBAN_ALL", "LGA_NAME_ALL", "REGION_NAME_ALL", "RMA", "RMA_ALL", "STAT_DIV_NAME"]].nunique()

ABS_CODE             3
ACCIDENT_STATUS      5
ALCOHOLTIME          2
ACCIDENT_TYPE        9
DCA_CODE            11
HIT_RUN_FLAG         3
LIGHT_CONDITION      7
POLICE_ATTEND        3
ROAD_GEOMETRY        9
SEVERITY             4
SPEED_ZONE          11
RUN_OFFROAD          2
NODE_TYPE            2
LGA_NAME            87
REGION_NAME          8
ALCOHOL_RELATED      2
DEG_URBAN_NAME       7
DEG_URBAN_ALL        8
LGA_NAME_ALL       215
REGION_NAME_ALL      8
RMA                  5
RMA_ALL             12
STAT_DIV_NAME        2
dtype: int64

In [19]:
#remove an empty row from region name
crashes_df = crashes_df.drop(crashes_df.loc[crashes_df['REGION_NAME'] == " "].index)

In [20]:
#convert day of the week to a string instead of an int, so that we can get dummies 
crashes_df['DAY_OF_WEEK'] = crashes_df['DAY_OF_WEEK'].astype('str')

In [21]:
#get dummies
crash_dummies = pd.get_dummies(crashes_df[["ABS_CODE", "ACCIDENT_STATUS", "ALCOHOLTIME", "ACCIDENT_TYPE", "DAY_OF_WEEK", "DCA_CODE", "HIT_RUN_FLAG", "LIGHT_CONDITION",
            "POLICE_ATTEND", "ROAD_GEOMETRY", "SEVERITY", "SPEED_ZONE", "RUN_OFFROAD", "NODE_TYPE", "REGION_NAME", 
            "ALCOHOL_RELATED", "DEG_URBAN_NAME", "DEG_URBAN_ALL", "REGION_NAME_ALL", "RMA", "RMA_ALL", "STAT_DIV_NAME"]], dtype=int)
#print out all the new column names
for column in crash_dummies.columns:
    print(column)

ABS_CODE_ABS to receive accident
ABS_CODE_Heart attk/suicide/death by nat.causes
ABS_CODE_Non ABS accident
ACCIDENT_STATUS_Discarded
ACCIDENT_STATUS_Finished
ACCIDENT_STATUS_Private Property
ACCIDENT_STATUS_Reopened
ACCIDENT_STATUS_Unfinished
ALCOHOLTIME_No
ALCOHOLTIME_Yes
ACCIDENT_TYPE_Collision with a fixed object
ACCIDENT_TYPE_Collision with vehicle
ACCIDENT_TYPE_Fall from or in moving vehicle
ACCIDENT_TYPE_No collision and no object struck
ACCIDENT_TYPE_Other accident
ACCIDENT_TYPE_Struck Pedestrian
ACCIDENT_TYPE_Struck animal
ACCIDENT_TYPE_Vehicle overturned (no collision)
ACCIDENT_TYPE_collision with some other object
DAY_OF_WEEK_1
DAY_OF_WEEK_2
DAY_OF_WEEK_3
DAY_OF_WEEK_4
DAY_OF_WEEK_5
DAY_OF_WEEK_6
DAY_OF_WEEK_7
DCA_CODE_CROSS TRAFFIC(INTERSECTIONS ONLY)
DCA_CODE_HEAD ON (NOT OVERTAKING)
DCA_CODE_LEFT OFF CARRIAGEWAY INTO OBJECT/PARKED VEHICLE
DCA_CODE_OFF RIGHT BEND INTO OBJECT/PARKED VEHICLE
DCA_CODE_OUT OF CONTROL ON CARRIAGEWAY (ON STRAIGHT)
DCA_CODE_Other
DCA_CODE_PED NEAR

In [22]:
crash_dummies.head()

,ABS_CODE_ABS to receive accident,ABS_CODE_Heart attk/suicide/death by nat.causes,ABS_CODE_Non ABS accident,ACCIDENT_STATUS_Discarded,ACCIDENT_STATUS_Finished,ACCIDENT_STATUS_Private Property,ACCIDENT_STATUS_Reopened,ACCIDENT_STATUS_Unfinished,ALCOHOLTIME_No,ALCOHOLTIME_Yes,...,"RMA_ALL_Arterial Other,Arterial Highway","RMA_ALL_Arterial Other,Local Road",RMA_ALL_Freeway,"RMA_ALL_Freeway,Arterial Other",RMA_ALL_Local Road,"RMA_ALL_Local Road,Arterial Highway","RMA_ALL_Local Road,Arterial Other",RMA_ALL_Other,STAT_DIV_NAME_Country,STAT_DIV_NAME_Metro
0,1,0,0,0,1,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0
1,1,0,0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,1
2,1,0,0,0,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
4,1,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
5,1,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,0


In [23]:
#get the numerical columns from the original data
crashes_cleaned = crashes_df[["ACCIDENT_DATE", "ACCIDENT_TIME", "LONGITUDE", "LATITUDE", "TOTAL_PERSONS", "INJ_OR_FATAL", 
                              "FATALITY", "SERIOUSINJURY", "OTHERINJURY", "NONINJURED", "MALES", "FEMALES", "BICYCLIST", 
                              "PASSENGER", "DRIVER", "PEDESTRIAN", "PILLION", "MOTORIST", "PED_CYCLIST_5_12", 
                              "PED_CYCLIST_13_18", "OLD_PEDESTRIAN", "OLD_DRIVER", "YOUNG_DRIVER", "UNLICENCSED", 
                              "NO_OF_VEHICLES", "HEAVYVEHICLE", "PASSENGERVEHICLE", "MOTORCYCLE", "PUBLICVEHICLE"]]

#merge with the new dummies columns
crashes_cleaned_df = pd.merge(crashes_cleaned, crash_dummies, left_index=True, right_index=True)
crashes_cleaned_df.head()

,ACCIDENT_DATE,ACCIDENT_TIME,LONGITUDE,LATITUDE,TOTAL_PERSONS,INJ_OR_FATAL,FATALITY,SERIOUSINJURY,OTHERINJURY,NONINJURED,...,"RMA_ALL_Arterial Other,Arterial Highway","RMA_ALL_Arterial Other,Local Road",RMA_ALL_Freeway,"RMA_ALL_Freeway,Arterial Other",RMA_ALL_Local Road,"RMA_ALL_Local Road,Arterial Highway","RMA_ALL_Local Road,Arterial Other",RMA_ALL_Other,STAT_DIV_NAME_Country,STAT_DIV_NAME_Metro
0,2015-07-01,5,144.368427,-38.177369,2,1,0,0,1,1,...,0,0,0,0,1,0,0,0,1,0
1,2015-07-01,11,145.079994,-37.656052,1,1,0,0,1,0,...,0,1,0,0,0,0,0,0,0,1
2,2015-07-01,13,144.894442,-37.816007,2,1,0,0,1,1,...,0,0,0,0,1,0,0,0,0,1
4,2015-07-02,10,143.251106,-38.015857,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
5,2015-07-02,8,146.486993,-38.188944,3,1,0,0,1,2,...,0,0,0,0,0,0,1,0,1,0


In [24]:
#export to a new csv
crashes_cleaned_df.to_csv("..\Resources\crashes_cleaned_df.csv")